In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random as rn
from matplotlib.patches import ConnectionPatch
import plotly as py

from scipy import stats
import regex as re


seed = 42

import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

In [2]:
email=pd.read_csv("spam.csv")
email.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
email.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
#pd.set_option("display.max_rows",5572)
email["Message"]

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object

In [5]:
email["spam"]=email["Category"].apply(lambda x: 1 if x=="spam" else 0)
email.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [6]:
email2=email.copy()

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import*
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [8]:
logistic=LogisticRegression()
tree=DecisionTreeClassifier()
forest=RandomForestClassifier()
ada=AdaBoostClassifier()
xgb=XGBClassifier()
#grid_search=GridSearchCV()
clf=MultinomialNB()
svm=SVC()

In [9]:
x1=email2["Message"]
y1=email2["spam"]
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,random_state=50,test_size=0.8)

In [10]:

vectorizer = CountVectorizer()
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])
clf.fit(x_train1,y_train1)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [11]:
encoder=LabelEncoder()
num_col=["Category","Message"]
pd.get_dummies(email,num_col,drop_first=True)
for col in num_col:
    email[col]=encoder.fit_transform(email[col])
email.head()

,Category,Message,spam
0,0,1080,0
1,0,3126,0
2,1,999,1
3,0,4121,0
4,0,2781,0


In [12]:
x=email["Message"]
y=email["spam"]
print(x.shape)
print(y.shape)

(5572,)
(5572,)


In [13]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=50,test_size=0.8)

In [14]:
print(f'X_train size= {x_train.shape}')
print(f'y_train size= {y_train.shape}')

x_train = x_train.values.reshape(-1, 1)
x_test = x_test.values.reshape(-1, 1)
print(f'X_train size= {x_train.shape}')
print(f'x_test size= {x_test.shape}')

print(x.shape)
print(y.shape)

X_train size= (1114,)
y_train size= (1114,)
X_train size= (1114, 1)
x_test size= (4458, 1)
(5572,)
(5572,)


In [15]:
def get_score(model):
    model.fit(x_train,y_train)
    pred=model.predict(x_test)
    accuracy=model.score(x_test,y_test)
    return accuracy


print(f' For Logistic Regression Algorithm, we had an accuracy of {get_score(logistic)}')
print(f' For  Decision Tree Algorithm, we had an accuracy of {get_score(tree)}')
print(f' For Random Forest Algorithm, we had an accuracy of {get_score(forest)}')
print(f' For xgboost Algorithm, we had an accuracy of {get_score(xgb)}')
print(f' For AdaBoostClassifier Algorithm, we had an accuracy of {get_score(ada)}')
print(f' For SVM Algorithm, we had an accuracy of {get_score(svm)}')
print(f' For Naive Bayes Algorithm, we had an accuracy of {clf.score(x_test1,y_test1)}')

 For Logistic Regression Algorithm, we had an accuracy of 0.8694481830417228
 For  Decision Tree Algorithm, we had an accuracy of 0.8660834454912517
 For Random Forest Algorithm, we had an accuracy of 0.8660834454912517
 For xgboost Algorithm, we had an accuracy of 0.8864961866307761
 For AdaBoostClassifier Algorithm, we had an accuracy of 0.8689995513683266
 For SVM Algorithm, we had an accuracy of 0.8694481830417228
 For Naive Bayes Algorithm, we had an accuracy of 0.9800358905338717


In [16]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf,x1,y1)


array([0.98295964, 0.98565022, 0.98294434, 0.98294434, 0.98653501])

In [17]:
x = x.values.reshape(-1, 1)

In [18]:
algorithms = [logistic, tree, forest, ada, xgb, svm]
algorithm_names = ["Logistic Regression", "Decision Tree",
                   "Random Forest", "AdaBoostClassifier",
                   "Xgboost", "SMV"]

# Loop through the algorithms and calculate and print accuracies
for col, col_name in zip(algorithms, algorithm_names):
    accuracies = cross_val_score(col, x, y)
    print(f'The accuracies for {col_name} are: {accuracies}')

The accuracies for Logistic Regression are: [0.86547085 0.86547085 0.86624776 0.86624776 0.86624776]
The accuracies for Decision Tree are: [0.8941704  0.91390135 0.91113106 0.91472172 0.9048474 ]
The accuracies for Random Forest are: [0.8941704  0.91390135 0.91113106 0.91472172 0.9048474 ]
The accuracies for AdaBoostClassifier are: [0.86457399 0.86547085 0.86445242 0.86624776 0.86624776]
The accuracies for Xgboost are: [0.91210762 0.91928251 0.92010772 0.90843806 0.91741472]
The accuracies for SMV are: [0.86547085 0.86547085 0.86624776 0.86624776 0.86624776]


In [28]:
cross_val_score(clf,x_test1,y_test1,cv=100).mean()

0.9872272727272728

Naive Bayes did a great job on this classification problem